In [ ]:
# 🔧 Cell 1
import mlflow, mlflow.sklearn
from pyspark.sql import SparkSession
from scripts.etl_vix import VIXETL
from scripts.anomaly_vix import VIXAnomalyDetector

spark = SparkSession.builder.appName("VIXPipeline").getOrCreate()

# 📦 Cell 2: ETL
VIXETL().run()

# 🧠 Cell 3: Anomaly Detection
with mlflow.start_run(run_name="VIX Anomaly"):
    det = VIXAnomalyDetector(spark, method="lof")
    anomalies = det.run()
    mlflow.log_param("method", det.method)
    mlflow.log_metric("num_anomalies", len(anomalies))
    mlflow.sklearn.log_model(det.model, "model")
    mlflow.log_artifact("scripts/anomaly_vix.py")

# 📊 Cell 4: Display
display(spark.table("main.etl.vix_monthly_agg"))
display(anomalies)

# 🔍 Cell 5: MLflow Tracking
print(f"Run ID: {mlflow.active_run().info.run_id}")
